# Wikipedia Kategorie:Juristische Fachzeitschrift (Deutschland) 

see https://de.wikipedia.org/w/index.php?title=Kategorie:Juristische_Fachzeitschrift_(Deutschland)

There is a [WikiData entitiy for this Wikipedia category (Q8979123)](https://www.wikidata.org/wiki/Q8979123) but as of 2024/12/11, there are no linked entities. 

With the help of https://petscan.wmcloud.org/, a [list of all journals of the category "Kategorie:Juristische Fachzeitschrift (Deutschland)"](data/wikipedia/category-law-journal-germany.csv) can be generated (including WikiData QIDs).

This then can be further transformed into a table with the URLs of the Wikipedia and WikiData pages:

In [ ]:
import csv
import requests

# Function to generate the Markdown table
def generate_markdown_table(csv_file):
    with open(csv_file, mode='r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        rows = list(reader)  # Read all rows into a list
        
        # Sort the rows by the 'title' column
        sorted_rows = sorted(rows, key=lambda x: x['title'])

        markdown_table = []

        # Add the header
        markdown_table.append("| Title | Wikipedia | WikiData |")
        markdown_table.append("|-------|-----------|----------|")

        for row in sorted_rows:
            # Replace underscores with spaces in the title
            title = row['title'].replace('_', ' ')

            # Wikipedia page
            if row['pageid'] != '':
                #print(f'Checking if Wikipedia page for {title} exists...')
                #wikipedia_api_url = f"https://de.wikipedia.org/w/api.php?action=query&pageids={row['pageid']}&format=json"
                #res = requests.get(wikipedia_api_url).json()
                #wikipedia_page_exists = res.get('query', {}).get('pages',{}).get(row['pageid'], {}).get('title', None)
                wikipedia_page_exists = True # We already checked that 
                wikipedia_url = f"https://de.wikipedia.org/w/?title={row['title']}"
                wikipedia_str = f'[Wikipedia]({wikipedia_url})' if wikipedia_page_exists else ''
            else:
                wikipedia_str = ""

            # Wikidata page
            if row['Wikidata'] != '':
                wikidata_url = f"https://www.wikidata.org/wiki/{row['Wikidata']}"
                wikidata_str = f'[WikiData]({wikidata_url})'
            else:
                wikidata_str = ''

            # Markdown row
            markdown_table.append(f"| {title} | {wikipedia_str} | {wikidata_str} |")

        return "\n".join(markdown_table)

with open('data/wikipedia/category-law-journal-germany.md', 'w', encoding='utf-8') as f:
    f.write(generate_markdown_table('data/wikipedia/category-law-journal-germany.csv'))


This list can be filtered to those entries which have an ISSN (retrieved via Wikidata), containing 147 entries

In [7]:
import pandas as pd
df_A = pd.read_csv('data\wikipedia\category-law-journal-germany.csv')
df_B = pd.read_csv('data\wikidata\cache.csv')
# Normalize title and journal columns
df_A['normalized_title'] = df_A['title'].str.lower().str.replace('_', ' ')
df_B['normalized_journal'] = df_B['journal'].str.lower()

# Merge DataFrames on the normalized columns
result_df = df_B.merge(df_A[['Wikidata', 'normalized_title']], 
                        left_on='normalized_journal', 
                        right_on='normalized_title', 
                        how='right')

# Discard non-matching rows 
result_df = result_df[result_df['journal'].notnull() & result_df['issn'].notnull() ]

# Select the desired columns and rename
result_df = result_df[['journal', 'Wikidata', 'abbreviation', 'issn']]
result_df = result_df.rename(columns={'Wikidata': 'qid'})
result_df.to_csv('data\wikipedia\category-law-journal-germany-with-issn.csv', index=False)
result_df

,journal,qid,abbreviation,issn
1,Der Staat,Q1197035,"STAAT, Staat, DS",0038-884X
2,Datenschutz und Datensicherheit,Q1172510,DuD,0724-4371
3,Gewerblicher Rechtsschutz und Urheberrecht,Q1520489,GRUR,0016-9420
5,Neue Juristische Wochenschrift,Q883935,NJW,0341-1915
6,Juristische Schulung,Q1713998,JuS,0022-6939
...,...,...,...,...
283,Das Grundeigentum,Q105826891,Grundeigentum,0017-4882
287,Zeitschrift für Rechtssoziologie,Q96335163,"ZfRsoz, Zs. f. Rechtssoz.",0174-0202
299,Zeitschrift für Lebensrecht,Q116235779,ZfL,0944-4521
301,NStZ-Rechtsprechungs-Report Strafrecht,Q117466106,NStZ-RR,0949-7129
